In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn.preprocessing import LabelBinarizer
import scipy.stats as stats
from sklearn import linear_model
from scipy.stats import ttest_rel

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [7]:
january_flight_data = pd.read_csv("..\January Flight Data.csv")
january_flight_data_df= january_flight_data[['Flight Date','Flight Day', 'Flight Month', 'Flight Year', 'Airline','ORIGIN','Destination','Departure Time','CRS Departure Time',
        'Taxi Out', 'Departure Delay', 'Air Time', 'Distance','Cancelled', 'Tail Number','ICAO Type','Manufacture Year','Temperature',
        'Cloud Cover','Active Weather']]

In [8]:
january_flight_data_df.head()

,Flight Date,Flight Day,Flight Month,Flight Year,Airline,ORIGIN,Destination,Departure Time,CRS Departure Time,Taxi Out,Departure Delay,Air Time,Distance,Cancelled,Tail Number,ICAO Type,Manufacture Year,Temperature,Cloud Cover,Active Weather
0,2022-01-01,1,1,2022,American Airlines,JFK,LAX,2022-01-01 06:57:00,2022-01-01 07:00:00,19,-3,347,2475,0,N106NN,A321,2014,9.39,4.0,0.0
1,2022-01-01,1,1,2022,American Airlines,LAX,JFK,2022-01-01 08:57:00,2022-01-01 09:00:00,20,-3,270,2475,0,N101NN,A321,2013,12.22,2.0,0.0
2,2022-01-01,1,1,2022,American Airlines,ORD,FLL,2022-01-01 09:37:00,2022-01-01 18:30:00,23,907,166,1182,0,N805NN,B738,2009,2.22,4.0,1.0
3,2022-01-01,1,1,2022,American Airlines,CLT,STL,2022-01-01 23:00:00,2022-01-01 22:40:00,16,20,104,575,0,N870NN,B738,2011,20.00,3.0,0.0
4,2022-01-01,1,1,2022,American Airlines,ATL,DFW,2022-01-01 16:24:00,2022-01-01 16:29:00,30,-5,114,731,0,N561UW,A321,2012,21.72,4.0,1.0


In [9]:
january_flight_data.describe()

,Taxi Out,Departure Delay,Air Time,Distance,Cancelled,Manufacture Year,Temperature,Cloud Cover,Active Weather,Flight Year,Flight Month,Flight Day
count,436603.000000,436603.000000,436603.000000,436603.000000,436603.000000,436603.000000,436603.000000,436603.000000,436603.000000,436603.0,436603.0,436603.000000
mean,17.149376,10.463240,113.108806,850.654643,0.069182,2009.463897,4.993400,2.145961,0.136369,2022.0,1.0,16.080616
std,10.539358,47.785129,74.722701,605.407442,0.344963,7.357847,9.753268,1.506643,0.355177,0.0,0.0,8.931706
min,0.000000,-55.000000,0.000000,66.000000,0.000000,1990.000000,-27.220000,0.000000,0.000000,2022.0,1.0,1.000000
25%,12.000000,-6.000000,60.000000,404.000000,0.000000,2003.000000,-1.720000,1.000000,0.000000,2022.0,1.0,8.000000
50%,15.000000,-2.000000,98.000000,717.000000,0.000000,2010.000000,4.390000,2.000000,0.000000,2022.0,1.0,16.000000
75%,20.000000,7.000000,146.000000,1089.000000,0.000000,2016.000000,11.720000,4.000000,0.000000,2022.0,1.0,24.000000
max,186.000000,2501.000000,727.000000,5095.000000,3.000000,2022.000000,31.720000,4.000000,2.000000,2022.0,1.0,31.000000


In [10]:
y = LabelBinarizer().fit_transform(january_flight_data_df["Active Weather"])

In [11]:
active_weather = pd.DataFrame(y,columns = ["No Weather","Moderate Weather", "Severe Weather"])

In [12]:
df = pd.concat([january_flight_data_df, active_weather],axis = 1, join = "inner")

In [13]:
df.head()

,Flight Date,Flight Day,Flight Month,Flight Year,Airline,ORIGIN,Destination,Departure Time,CRS Departure Time,Taxi Out,...,Cancelled,Tail Number,ICAO Type,Manufacture Year,Temperature,Cloud Cover,Active Weather,No Weather,Moderate Weather,Severe Weather
0,2022-01-01,1,1,2022,American Airlines,JFK,LAX,2022-01-01 06:57:00,2022-01-01 07:00:00,19,...,0,N106NN,A321,2014,9.39,4.0,0.0,1,0,0
1,2022-01-01,1,1,2022,American Airlines,LAX,JFK,2022-01-01 08:57:00,2022-01-01 09:00:00,20,...,0,N101NN,A321,2013,12.22,2.0,0.0,1,0,0
2,2022-01-01,1,1,2022,American Airlines,ORD,FLL,2022-01-01 09:37:00,2022-01-01 18:30:00,23,...,0,N805NN,B738,2009,2.22,4.0,1.0,0,1,0
3,2022-01-01,1,1,2022,American Airlines,CLT,STL,2022-01-01 23:00:00,2022-01-01 22:40:00,16,...,0,N870NN,B738,2011,20.00,3.0,0.0,1,0,0
4,2022-01-01,1,1,2022,American Airlines,ATL,DFW,2022-01-01 16:24:00,2022-01-01 16:29:00,30,...,0,N561UW,A321,2012,21.72,4.0,1.0,0,1,0


# Explanation

With our data frame create, we run a groupby, grouping by Airports, so that we can see the how each airport's details. We need to then build out the measures to view and combine them into a dataframe to run our tests.

In [32]:
origin_caused_delay = df.groupby("ORIGIN")

In [33]:
origin_delay = origin_caused_delay['ORIGIN','Air Time','Distance','Departure Delay','Cloud Cover']

c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [34]:
origin_delay = origin_delay.mean()

In [35]:
no_active_weather = df[(df['No Weather'] == 1)].groupby('ORIGIN')['No Weather'].sum()

In [36]:
moderate_active_weather = df[(df['Moderate Weather'] == 1)].groupby('ORIGIN')['Moderate Weather'].sum()

In [37]:
severe_active_weather = df[(df['Severe Weather'] == 1)].groupby('ORIGIN')['Severe Weather'].sum()

In [38]:
flight_count = origin_caused_delay['ORIGIN'].count()

In [39]:
origin_delay['Total Flights in January'] = flight_count

In [40]:
origin_delay["Flights per Day"] = origin_delay["Total Flights in January"]/31

In [41]:
origin_delay['No Active Weather'] = no_active_weather

In [42]:
origin_delay['Moderate Weather'] = moderate_active_weather

In [43]:
origin_delay['Severe Weather'] = severe_active_weather

In [44]:
origin_delay["Total Taxi Out Time"] = origin_caused_delay["Taxi Out"].sum()
origin_delay["Taxi Time Per Flight"] = origin_delay["Total Taxi Out Time"] / origin_delay["Total Flights in January"] 

In [45]:
origin_delays = origin_delay.fillna(0).sort_values(by = "Departure Delay", ascending = False)

In [46]:
origin_delays.head()

,Air Time,Distance,Departure Delay,Cloud Cover,Total Flights in January,Flights per Day,No Active Weather,Moderate Weather,Severe Weather,Total Taxi Out Time,Taxi Time Per Flight
ORIGIN,,,,,,,,,,,
EYW,108.944444,823.380051,30.441919,1.554293,792,25.548387,768,23.0,1.0,10755,13.579545
JAC,101.800388,796.160853,28.740310,1.531008,516,16.645161,360,153.0,3.0,12145,23.536822
STT,184.649194,1369.695565,26.129032,1.237903,496,16.000000,492,4.0,0.0,7238,14.592742
DEN,105.947547,851.465588,18.921105,2.074742,20952,675.870968,17708,2863.0,381.0,378472,18.063765
ROC,63.766384,398.164996,17.696993,3.296068,1297,41.838710,700,591.0,6.0,26274,20.257517


With our dataframe built, we can run describe() to get a general desrciption of our columns.

In [47]:
origin_delays.describe()

,Air Time,Distance,Departure Delay,Cloud Cover,Total Flights in January,Flights per Day,No Active Weather,Moderate Weather,Severe Weather,Total Taxi Out Time,Taxi Time Per Flight
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,108.746662,807.424977,10.612840,1.983762,6063.930556,195.610663,5262.402778,776.125000,25.402778,103992.625000,15.879680
std,28.769751,251.606959,5.228199,0.611168,5999.190916,193.522288,5323.012492,1046.285205,58.126782,112794.528466,3.021507
min,63.029979,385.706638,3.168251,0.330490,108.000000,3.483871,103.000000,0.000000,0.000000,1405.000000,11.295366
25%,90.989200,641.094019,7.492193,1.616067,1589.750000,51.282258,1512.250000,126.250000,0.000000,24210.250000,13.501429
50%,102.107903,750.153824,9.745825,2.063249,3539.500000,114.177419,3075.000000,368.000000,6.000000,54022.000000,15.159447
75%,122.183574,947.602315,12.359226,2.426629,9730.250000,313.879032,7419.250000,975.250000,21.000000,173819.750000,17.855430
max,188.898191,1574.472932,30.441919,3.296068,25277.000000,815.387097,22349.000000,5886.000000,381.000000,497040.000000,23.536822


Now we can run our tests. We first run a Multi-Linear regression so we can figure out which column is most influential in causing departure delays and after that, we can verify that it affects the dependent variable by running a t test and confirming it against the Null Hypothesis of zero affect.

# Multi-Linear Regression

In [49]:
X = origin_delays[['Air Time', 'Distance', 'Cloud Cover','No Active Weather', 'Moderate Weather',
                   'Severe Weather', 'Flights per Day', 'Taxi Time Per Flight']]
y = origin_delays['Departure Delay']

regr = linear_model.LinearRegression()
regr.fit(X, y)

print(regr.coef_)

[ 1.03668533e-01 -4.68515717e-03 -9.73619886e-01 -3.33667084e-04
 -5.87267110e-04  2.17620457e-02  6.72293921e-04  1.03011741e+00]


A Multi-Linear works by building out an equation for a line, much like the single linear regression but it uses multiple slopes to find the variable that impacts the y variable (Departure Delays in this case) the most. In our case, each variable seems to impact Departure Delays only a slight amount so that would lead us to think that the main cause of flight delays are something else entirely outside of our dataset. But in this case, the Taxi Time Per Flight seems to be the main cause. So we can take Taxi Flight Time and use it in our hypotheisis.

# Null Hypothesis
Taxi Flight Times have no affect on Flight Delays

# t-Test

In [50]:
#group1 = origin_delays[origin_delays['Departure Delay']]
#group2 = origin_delays[origin_delays['Taxi Time Per Flight']]
stats.ttest_ind(origin_delays['Taxi Time Per Flight'], origin_delays['Departure Delay'],equal_var = False)

Ttest_indResult(statistic=7.400937226766764, pvalue=2.5218024808967977e-11)

# Conclusion
With a p-value of less than .05, we fail to reject the null hypothesis, meaning that Taxi Time Per Flight has an affect on Departure Delays.